In [2]:
#download libraries and packages
# !pip install imutils
# !pip install opencv-python
# !pip install tensorflow
# !pip install tensorflow_hub
# !pip install matplotlib

from imutils import paths
# from PIL import Image
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import random
import os
import tensorflow_hub as hub

In [3]:
batch_size = 16
img_height = 200 
img_width = 200 
training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/cs/student/msc/aisd/2022/yintimun/Downloads/train', # directory path
    validation_split = None, 
    subset = None, 
    image_size = (img_height, img_width), # image size of height and width
    batch_size = batch_size # batch size
)

Found 28709 files belonging to 7 classes.


2023-08-17 11:08:03.689804: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-17 11:08:03.771180: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
testing_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/cs/student/msc/aisd/2022/yintimun/Downloads/test', # directory path
    validation_split = None, 
    subset = None, 
    image_size = (img_height, img_width), # image size of height and width
    batch_size = batch_size # batch size
)

Found 7178 files belonging to 7 classes.


In [5]:
# One-hot encode the labels
num_classes = len(training_ds.class_names)
def one_hot_encode(image, label):
    label = tf.one_hot(label, num_classes)
    return image, label

# # Apply one-hot encoding to the dataset
training_ds = training_ds.map(one_hot_encode)
testing_ds = testing_ds.map(one_hot_encode)

In [6]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D



img_channels = 3
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, img_channels))
for layer in base_model.layers:
    layer.trainable = False

effnet = Sequential()
effnet.add(base_model)
effnet.add(GlobalAveragePooling2D())
effnet.add(Dense(256, activation='relu'))
effnet.add(Dense(7, activation='softmax'))
effnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_saved = effnet.fit(training_ds, validation_data=testing_ds, epochs=20)
# effnet.save("/Users/janrauhl/Documents/Dissertation/FER/efficientnet_base.h5")

Epoch 1/20
1795/1795 [==============================] - 209s 114ms/step - loss: 1.4240 - accuracy: 0.4556 - val_loss: 1.3020 - val_accuracy: 0.5000
Epoch 2/20
1795/1795 [==============================] - 204s 114ms/step - loss: 1.2841 - accuracy: 0.5087 - val_loss: 1.2630 - val_accuracy: 0.5223
Epoch 3/20
1676/1795 [===========================>..] - ETA: 10s - loss: 1.2181 - accuracy: 0.5412

KeyboardInterrupt: 

In [ ]:
#CNN model
CNN = tf.keras.models.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255), 
  layers.Conv2D(32, 3, activation = 'relu'),  # convolution layers
  layers.MaxPooling2D(), # maximum pooling layers
  layers.Conv2D(32, 3, activation = 'relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation = 'relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 3, activation = 'relu'),
  layers.MaxPooling2D(),
  layers.GlobalAveragePooling2D(),
  layers.Dense(512, activation = 'relu'), # dense layer
  layers.Dense(9, activation =  'softmax') # soft-max layer
])

#CNN with adam optimiser, sparse categorical crossentropy loss, and accuracy metrics
CNN.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])
#fit model to training and validation data, with 20 epochs
!pip install h5py

########## ONLY RUN ONCE
model_saved = CNN.fit(training_ds, validation_data= testing_ds, epochs = 20)